Monday, November 6, 2023

DO NOT MODIFY OR DELETE THIS NOTEBOOK! I WANT TO RETAIN THE CELL OUTPUT!

'meta-llama/Llama-2-13b-hf'

Friday, November 3, 2023

https://github.com/facebookresearch/llama-recipes/blob/main/examples/quickstart.ipynb

So it turns out I cannot run this notebook in isolation because it has dependencies on other modules. So looks like I am gonna run this from there, just to see if it works.

Whelp, actually, you can .. just gotta set the current working directory ... (duh!)

Copyright (c) Meta Platforms, Inc. and affiliates.
This software may be used and distributed according to the terms of the Llama 2 Community License Agreement.

## Quick Start Notebook

This notebook shows how to train a Llama 2 model on a single GPU (e.g. A10 with 24GB) using int8 quantization and LoRA.

### Step 0: Install pre-requirements and convert checkpoint

The example uses the Hugging Face trainer and model which means that the checkpoint has to be converted from its original format into the dedicated Hugging Face format.
The conversion can be achieved by running the `convert_llama_weights_to_hf.py` script provided with the transformer package.
Given that the original checkpoint resides under `models/7B` we can install all requirements and convert the checkpoint with:

In [ ]:
# %%bash
# pip install llama-recipes transformers datasets accelerate sentencepiece protobuf==3.20 py7zr scipy peft bitsandbytes fire torch_tb_profiler ipywidgets
# TRANSFORM=`python -c "import transformers;print('/'.join(transformers.__file__.split('/')[:-1])+'/models/llama/convert_llama_weights_to_hf.py')"`
# python ${TRANSFORM} --input_dir models --model_size 7B --output_dir models_hf/7B

### Step 1: Load the model

Point model_id to model weight folder

In [1]:
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer

model_id="./models_hf/7B"

# override the default with a llama2 model that I do have ... will this work?? Yup!
model_id = 'meta-llama/Llama-2-7b-hf' # This works!
model_id = 'meta-llama/Llama-2-7b-chat-hf'
model_id = 'meta-llama/Llama-2-13b-hf' # Wow! This also works! 23,734 MiB
model_id = 'meta-llama/Llama-2-13b-chat-hf'

# Last guy wins, right?! ...
model_id = 'meta-llama/Llama-2-13b-hf'

tokenizer = LlamaTokenizer.from_pretrained(model_id)

# original code ...
# model =LlamaForCausalLM.from_pretrained(model_id, load_in_8bit=True, device_map='auto', torch_dtype=torch.float16)

In [2]:
%%time

model =LlamaForCausalLM.from_pretrained(model_id, load_in_8bit=True, torch_dtype=torch.float16)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

CPU times: user 5.74 s, sys: 8.55 s, total: 14.3 s
Wall time: 15.7 s


### Step 2: Load the preprocessed dataset

We load and preprocess the samsum dataset which consists of curated pairs of dialogs and their summarization:

In [3]:
# set the current working directory to where the llama-recipes code resides ...
import os

# but first, grab the current working direcory ...
currentWorkingDirectory = os.getcwd()
print(currentWorkingDirectory)

# and now switch to the llama-recipes source directory ...
os.chdir('/tf/All/Data/Documents/Github/facebookresearch/llama-recipes/src')

/tf/All/Data/Documents/Github/rkaunismaa/LLaMa2_PlayGround


In [4]:
from llama_recipes.utils.dataset_utils import get_preprocessed_dataset
from llama_recipes.configs.datasets import samsum_dataset

train_dataset = get_preprocessed_dataset(tokenizer, samsum_dataset, 'train')

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

In [5]:
# change back to the original current working directory
os.chdir(currentWorkingDirectory)

To understand what the samsum dataset is about, go [here](https://huggingface.co/datasets/samsum)

The SAMSum dataset contains about 16k messenger-like conversations with summaries. Conversations were created and written down by linguists fluent in English. Linguists were asked to create conversations similar to those they write on a daily basis, reflecting the proportion of topics of their real-life messenger convesations. The style and register are diversified - conversations could be informal, semi-formal or formal, they may contain slang words, emoticons and typos. Then, the conversations were annotated with summaries. It was assumed that summaries should be a concise brief of what people talked about in the conversation in third person. The SAMSum dataset was prepared by Samsung R&D Institute Poland and is distributed for research purposes (non-commercial licence: CC BY-NC-ND 4.0).

### Step 3: Check base model

Run the base model on an example input:

In [6]:
eval_prompt = """
Summarize this dialog:
A: Hi Tom, are you busy tomorrow’s afternoon?
B: I’m pretty sure I am. What’s up?
A: Can you go with me to the animal shelter?.
B: What do you want to do?
A: I want to get a puppy for my son.
B: That will make him so happy.
A: Yeah, we’ve discussed it many times. I think he’s ready now.
B: That’s good. Raising a dog is a tough issue. Like having a baby ;-) 
A: I'll get him one of those little dogs.
B: One that won't grow up too big;-)
A: And eat too much;-))
B: Do you know which one he would like?
A: Oh, yes, I took him there last Monday. He showed me one that he really liked.
B: I bet you had to drag him away.
A: He wanted to take it home right away ;-).
B: I wonder what he'll name it.
A: He said he’d name it after his dead hamster – Lemmy  - he's  a great Motorhead fan :-)))
---
Summary:
"""

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()

with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))


Summarize this dialog:
A: Hi Tom, are you busy tomorrow’s afternoon?
B: I’m pretty sure I am. What’s up?
A: Can you go with me to the animal shelter?.
B: What do you want to do?
A: I want to get a puppy for my son.
B: That will make him so happy.
A: Yeah, we’ve discussed it many times. I think he’s ready now.
B: That’s good. Raising a dog is a tough issue. Like having a baby ;-) 
A: I'll get him one of those little dogs.
B: One that won't grow up too big;-)
A: And eat too much;-))
B: Do you know which one he would like?
A: Oh, yes, I took him there last Monday. He showed me one that he really liked.
B: I bet you had to drag him away.
A: He wanted to take it home right away ;-).
B: I wonder what he'll name it.
A: He said he’d name it after his dead hamster – Lemmy  - he's  a great Motorhead fan :-)))
---
Summary:
* 100%





We can see that the base model only repeats the conversation.

### Step 4: Prepare model for PEFT

Let's prepare the model for Parameter Efficient Fine Tuning (PEFT):

In [7]:
model.train()

def create_peft_config(model):
    
    from peft import (
        get_peft_model,
        LoraConfig,
        TaskType,
        prepare_model_for_int8_training,
    )

    peft_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        inference_mode=False,
        r=8,
        lora_alpha=32,
        lora_dropout=0.05,
        target_modules = ["q_proj", "v_proj"]
    )

    # prepare int-8 model for training
    model = prepare_model_for_int8_training(model)
    model = get_peft_model(model, peft_config)

    model.print_trainable_parameters()
    
    return model, peft_config

# create peft config
model, lora_config = create_peft_config(model)



/usr/local/lib/python3.8/dist-packages/peft/utils/other.py:135: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


trainable params: 6,553,600 || all params: 13,022,417,920 || trainable%: 0.05032552357220002


### Step 5: Define an optional profiler

In [8]:
from transformers import TrainerCallback
from contextlib import nullcontext

enable_profiler = False
# my override of the above ... nope! still getting errors!
# enable_profiler = True

output_dir = "tmp/llama-output"
# reset the name according to which model is being used
output_dir = "tmp/llama-output-13B-HF"

config = {
    'lora_config': lora_config,
    'learning_rate': 1e-4,
    'num_train_epochs': 1,
    'gradient_accumulation_steps': 2,
    'per_device_train_batch_size': 2,
    'gradient_checkpointing': False,
}

# Set up profiler
if enable_profiler:
    wait, warmup, active, repeat = 1, 1, 2, 1
    total_steps = (wait + warmup + active) * (1 + repeat)
    schedule =  torch.profiler.schedule(wait=wait, warmup=warmup, active=active, repeat=repeat)
    profiler = torch.profiler.profile(
        schedule=schedule,
        on_trace_ready=torch.profiler.tensorboard_trace_handler(f"{output_dir}/logs/tensorboard"),
        record_shapes=True,
        profile_memory=True,
        with_stack=True)
    
    class ProfilerCallback(TrainerCallback):
        def __init__(self, profiler):
            self.profiler = profiler
            
        def on_step_end(self, *args, **kwargs):
            self.profiler.step()

    profiler_callback = ProfilerCallback(profiler)
else:
    profiler = nullcontext()

### Step 6: Fine tune the model

Here, we fine tune the model for a single epoch which takes a bit more than an hour on a A100.

In [9]:
from transformers import default_data_collator, Trainer, TrainingArguments

# Define training args
training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    bf16=True,  # Use BF16 if available
    # logging strategies
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=10,
    save_strategy="no",
    optim="adamw_torch_fused",
    max_steps=total_steps if enable_profiler else -1,
    **{k:v for k,v in config.items() if k != 'lora_config'}
)

# with profiler:
#     # Create Trainer instance
#     trainer = Trainer(
#         model=model,
#         args=training_args,
#         train_dataset=train_dataset,
#         data_collator=default_data_collator,
#         callbacks=[profiler_callback] if enable_profiler else [],
#     )

    # Start training
    # trainer.train()

The next cell was added as a workaround to get this notebook to train. This suggestion can be seen [here](https://github.com/facebookresearch/llama-recipes/issues/285)

In [10]:
from llama_recipes.data.concatenator import ConcatDataset

context_length = 2048 # CUDA out of memory issues!

context_length = 1024

train_batched = ConcatDataset(train_dataset, chunk_size=context_length)


Preprocessing dataset: 100%|██████████| 14732/14732 [00:03<00:00, 4614.45it/s]


In [11]:
# with profiler:
#     # Create Trainer instance
#     trainer = Trainer(
#         model=model,
#         args=training_args,
#         train_dataset=train_dataset,
#         data_collator=default_data_collator,
#         callbacks=[profiler_callback] if enable_profiler else [],
#     )

# This does nothing if profile is nullcontext() ... 

with profiler:
    # Create Trainer instance
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_batched, # notice we use train_batched, not train_dataset
        data_collator=default_data_collator,
        callbacks=[profiler_callback] if enable_profiler else [],
    )

CODECARBON : No CPU tracking mode found. Falling back on CPU constant mode.


In [12]:
%%time
# Start training
trainer.train()

# 'meta-llama/Llama-2-13b-hf'
# CPU times: user 1h 11min 6s, sys: 11.4 s, total: 1h 11min 18s
# Wall time: 1h 11min 21s

# 'meta-llama/Llama-2-13b-hf'
# CPU times: user 1h 11min 5s, sys: 7.65 s, total: 1h 11min 13s
# Wall time: 1h 11min 20s

# 'meta-llama/Llama-2-13b-hf'
# 71m, 19.9s

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.8/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.8/dist-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization


Step,Training Loss
10,1.555900
20,1.301800
30,1.198800
40,1.173900
50,1.180100
60,1.131400
70,1.138600
80,1.207300
90,1.168600
100,1.142700


CPU times: user 1h 11min 6s, sys: 11.4 s, total: 1h 11min 18s
Wall time: 1h 11min 21s


TrainOutput(global_step=388, training_loss=1.1496254414627232, metrics={'train_runtime': 4277.8016, 'train_samples_per_second': 0.725, 'train_steps_per_second': 0.091, 'total_flos': 2.4506762470096896e+17, 'train_loss': 1.1496254414627232, 'epoch': 1.0})

### Step 7:
Save model checkpoint

In [12]:
model.save_pretrained(output_dir)

### Step 8:
Try the fine tuned model on the same example again to see the learning progress:

In [13]:
model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))



Summarize this dialog:
A: Hi Tom, are you busy tomorrow’s afternoon?
B: I’m pretty sure I am. What’s up?
A: Can you go with me to the animal shelter?.
B: What do you want to do?
A: I want to get a puppy for my son.
B: That will make him so happy.
A: Yeah, we’ve discussed it many times. I think he’s ready now.
B: That’s good. Raising a dog is a tough issue. Like having a baby ;-) 
A: I'll get him one of those little dogs.
B: One that won't grow up too big;-)
A: And eat too much;-))
B: Do you know which one he would like?
A: Oh, yes, I took him there last Monday. He showed me one that he really liked.
B: I bet you had to drag him away.
A: He wanted to take it home right away ;-).
B: I wonder what he'll name it.
A: He said he’d name it after his dead hamster – Lemmy  - he's  a great Motorhead fan :-)))
---
Summary:
 A wants to get a puppy for his son. They've been talking about it for a long time. A wants to get a small dog. B is willing to go to the animal shelter with A tomorrow.
